In [ ]:
../julia-1.10.0/bin/julia --project=. --thread 32

In [ ]:
## single family learning
using Revise, HopfieldDCA, JLD2

f = 14; H = 21; n_ep = 150; alg = HopPlmVar_gen(H, seq_paths_dict[f], T = Float32); η = 0.5; λ = 0.001;  @time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f],
    orthog = false,
    fig_save = false, 
    par_save = false);

#fixed V
@load "../mj_eigen_with_gap.jld2";
f = 14; H = 21;  V = v; n_ep = 11; alg = HopPlmVar_gen(H, seq_paths_dict[f], 
    T = Float32); η = 0.05; λ = Float32(0.001
);  @time m,m_anal,n = trainer_J_fixedV(alg, n_ep, V, η = η, λ = λ, 
           structfile = structs_dict[f],
           folder = folders_dict[f], 
           savefile = folders_dict[f], 
           fig_save = true, 
           par_save = false);

#given V


f = 14; H = 20; n_ep = 101;alg = HopPlmVar_gen(H, seq_paths_dict[f], T = Float32); η = 0.05; λ = Float32(0.001);  @time m,n = trainer_J_givenV(alg, n_ep, V, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = true, 
    par_save = true);

#fixed V on subsample

f = 105; H = 20; n_ep = 150; n_seq = 200; alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.05; λ = Float32(0.001);  @time m,n = trainer_J_fixedV(alg, n_ep, V, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = false, 
    par_save = false);

#fixed K on subsample

f = 105; H = 21; n_ep = 150; n_seq = 200; alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.05; λ = Float32(0.001);  @time m,n = 
HopfieldDCA.trainer_J_fixedK(alg, n_ep, K, V, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = true, 
    par_save = true);


#full Hopfield

f = 14; H = 200; n_ep = 110; alg = HopPlmVar_full(H, seq_paths_dict[f], T = Float32); η = 0.5; λ = 0.001;  @time m,n = trainer_fullJ(alg, n_ep, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f],
    orthog = false,
    fig_save = false, 
    par_save = false);



f = 13354; H = 20; n_ep = 2; alg = HopPlmVar_full(H, seq_paths_dict[f], T = Float32); η = 0.5; λ = 0.001;  @time m,n = trainer_fullJ(alg, n_ep, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = true, 
    par_save = true);


In [ ]:
using Revise, HopfieldDCA, JLD2, DCAUtils, Optim, Statistics, Tullio, PyPlot, LinearAlgebra


# for every ij
@load "../mj_eigen_with_gap.jld2";
fam = 14; H = 21;  N = 53;V = v; 
Z = read_fasta_alignment(seq_paths_dict[fam], 0.9);alpha = 0.5;
@time K, V_new = HopfieldDCA.minimize_opt_loss_ij(Int.(Z), V, N, H, alpha = alpha, optimizer=LBFGS(), tol = 1e-18);
c_emp, c_emp_inv, c_mf, c_mf_inv = HopfieldDCA.get_corr_mf(Int.(Z), K, V_new, N, H-1) ;
cor(c_emp[:], c_mf[:])
cor(c_emp_inv[:], c_mf_inv[:])

#=using Optim
i = 10; j = 15;
K0= rand(N,N,20);
c_inv_ij = reshape(c_emp_inv, 20, N, 20, N)[:,i,:,j] 

           
K_vec_init = randn(20);    
objective(K_vec) = HopfieldDCA.opt_loss_ij(c_inv_ij, K_vec, Float64.(V_new))
result = optimize(objective, K_vec_init, ParticleSwarm(), Optim.Options(f_tol = 10^-18));
=#

close("all"); plt.scatter(c_emp[:], c_mf[:], label = "c_ij"); plt.legend(
    );plt.xlabel("Empirical"); plt.ylabel("Mean field");savefig("../c_ij.png")


close("all"); plt.scatter(c_emp_inv[:], c_mf_inv[:], label = "c_ij^-1"); plt.legend(
    );plt.xlabel("Empirical"); plt.ylabel("Mean field");savefig("../c_ij^-1.png")


l_emp, v_emp = eigen(c_emp);
l_mf, v_mf = eigen(c_mf);

l_emp_inv, v_emp_inv = eigen(c_emp_inv);
l_mf_inv, v_mf_inv = eigen(c_mf_inv);

close("all"); plt.hist(l_emp[1:end], bins = 50, histtype = "step", label = "Empirical", density = true
    ); plt.hist(l_mf[1:end], bins = 50, histtype = "step", label = "Mean field", density = true); plt.xlabel("Eigenvals of C_ij"
    ); plt.legend();savefig("../eigen_c_ij.png")

close("all"); plt.hist(l_emp_inv[1:end], bins = 50, histtype = "step", label = "Empirical", density = true
    ); plt.hist(l_mf_inv[1:end], bins = 50, histtype = "step", label = "Mean field", density = true); plt.xlabel("Eigenvals of C_ij^-1"
    ); plt.legend();savefig("../eigen_c_ij^-1.png")


@time s_anal = HopfieldDCA.score(K, V_new); 
PPV_h_anal = HopfieldDCA.compute_PPV(s_anal, structs_dict[fam]);
s = HopfieldDCA.score(K, V_new); 
PPV = HopfieldDCA.compute_PPV(s, structs_dict[fam]);
println(PPV_h_anal[53])
folder = folders_dict[fam];
@load joinpath(folder, "PPV_s.jld2") ss
@load joinpath(folder, "PPV_plm.jld2") splm
fig,ax1 = plt.subplots();ax1.plot(ss, label="Stucture");ax1.plot(splm, 
    label="PLM");ax1.plot(PPV_h_anal, label="mfHop");ax1.set_xscale("log");ax1.set_xlabel(
    "Number of Predictions");ax1.set_ylabel("PPV");ax1.legend();savefig("../test_ppv_pseudocount.png")



In [ ]:
fs = [169, 595, 677, 13354]; n_eps = [150, 150, 250, 250]
for i in 1:length(fs)
    f = fs[i]
    H = 256; n_ep = n_eps[i]; alg = HopPlmVar_full(H, seq_paths_dict[f], T = Float32); η = 0.2; λ = 0.01;  @time m,n = trainer_fullJ(alg, n_ep, η = η, λ = λ, 
           structfile = structs_dict[f],
           folder = folders_dict[f], 
           savefile = folders_dict[f],
           orthog = false,
           fig_save = true, 
           par_save = true);
end


fs = [677, 763, 13354]; n_eps = [700, 700, 700]
for i in 1:length(fs)
    f = fs[i]
    H = 128; n_ep = n_eps[i]; alg = HopPlmVar_full(H, seq_paths_dict[f], T = Float32); η = 0.2; λ = 0.001;  @time m,n = trainer_fullJ(alg, n_ep, η = η, λ = λ, 
           structfile = structs_dict[f],
           folder = folders_dict[f], 
           savefile = folders_dict[f],
           orthog = false,
           fig_save = true, 
           par_save = true);
end

In [ ]:
fs = [35, 169, 595, 677, 763, 13354]; n_eps = [250, 100, 250, 350, 450, 450]
for i in 1:length(fs)
    f = fs[i]; println("FAM : $(f)");
    H = 128; n_ep = n_eps[i]; alg = HopPlmVar_full(H, seq_paths_dict[f], T = Float32);
    
end

In [ ]:
#multi family learning
using Revise, HopfieldDCA

#semi_hopfield francesco family

#fixed V
fs = [14, 35, 76, 677, 763]; H = 21; algs = [HopPlmVar_gen(H, seq_paths_dict[f], T = Float32) 
    for f in fs]; 
n_ep = 2; η = 0.5; lambda = Float32(0.001); @time Ks = multitrainer_fixedV(fs, algs, n_ep, V, η = η, lambda = lambda,
savefile = "../ciao.txt", fig_save = true);




#normal
fs = [14, 35, 76, 677, 763]; H = 20; algs = [HopPlmVar_gen(H, seq_paths_dict[f], T = Float32) 
    for f in fs]; 
n_ep = 70; η = 0.05; lambda = 0.001; @time m = multitrainer(fs, algs, n_ep, 
    η = η, lambda = lambda, savefile = "../ciao.txt", 
    fig_save = true, par_save = true, orthog = false, 
    each_step = 10);

#full_hopfield francesco family

fs = [14, 35, 76, 677, 763]; H = 128; algs = [HopPlmVar_full(H, seq_paths_dict[f], T = Float32) 
    for f in fs]; 
n_ep = 70; η = 0.5; lambda = 0.001; @time m = multitrainer_full(fs, algs, n_ep, 
    η = η, lambda = lambda, savefile = "../ciao.txt", 
    fig_save = true, par_save = true, orthog = false, 
    each_step = 10);



In [ ]:
#multi family learning
using Revise, HopfieldDCA
fs = [14, 90, 105, 131, 200, 412, 593, 1774, 1807, 2953, 7648]; H = 21; algs = [HopPlmVar_gen(H, seq_paths_dict[f], T = Float32) 
    for f in fs]; 
n_ep = 20; η = 0.05; lambda = 0.001; @time m = multitrainer(fs, algs, n_ep, 
    η = η, lambda = lambda, savefile = "../ciao.txt", 
    fig_save = true, par_save = true, orthog = false, 
    each_step = 5, ort = 50);



In [ ]:
## performance on small alignments
using Revise, HopfieldDCA, JLD2, DCAUtils, Optim, PlmDCA


@load "../mj_eigen_with_gap.jld2";

fam = 14; H = 21;  N = 53;V = v; 

ppvs = []
f = 14; H = 21; n_ep = 300;  Zs = [100, 500, 1000]; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.5; λ = 0.001;  @time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f],
    orthog = false,
    fig_save = false, 
    par_save = false);
    s = score(m.K, m.V); ppv = compute_PPV(s, structs_dict[f]);
    push!(ppvs, ppv)
end

ppvs_fixed_V = []
f = 14; H = 21; n_ep = 100;  Zs = [100, 500, 1000]; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.05; λ = Float32.(0.001);  @time m,m_anal,n = trainer_J_fixedV(alg, n_ep, V, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = false, 
    par_save = false);
    s = score(m.K, V); ppv = compute_PPV(s, structs_dict[f]);
    push!(ppvs_fixed_V, ppv)
end

ppvs_mf = []; alpha = 0.5; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); @time K, V_new = HopfieldDCA.minimize_opt_loss_ij(Int.(alg.Z),
        V, N, H, alpha = alpha);
    s = score(K, V_new); ppv = compute_PPV(s, structs_dict[f]);
    push!(ppvs_mf, ppv)
end

ppvs_plm = []; f = 14; H = 21; n_ep = 200;  Zs = [100, 500, 1000]; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.5; λ = Float32.(0.001);  
    res = plmdca_asym(alg.Z, Float64.(alg.W) ./ sum(Float64.(alg.W)), lambdaJ=0.01,lambdaH=0.01,epsconv=1e-12);
    ppv_plm = compute_PPV(res.score, structs_dict[f]);
    push!(ppvs_plm, ppv_plm)    
end

for i in 1:length(Zs)
    fig,ax1 = plt.subplots() 
    ax1.plot(ppvs[i], label="Hop")
    ax1.plot(ppvs_fixed_V[i], label="Hop fix V")
    ax1.plot(ppvs_mf[i], label = "Hop mf")
    ax1.plot(ppvs_plm[o], label = "Plm")
    ax1.set_xscale("log")
    ax1.set_xlabel("Number of Predictions")
    ax1.set_ylabel("PPV")
    ax1.legend()
    savefig("../F$(f)Z$(Zs[i]).png"); 
end


fig,ax1 = plt.subplots(); ax1.plot(splm, label="Plm"); ax1.plot(
    ppv_ar_hop, label="Full_Hop 2%"); ax1.set_xscale("log"); ax1.set_xlabel(
    "Number of Predictions"); ax1.set_ylabel("PPV"); ax1.legend(); savefig("../f14_full_hop_2%ppv.png")

In [ ]:
out_path ="../multi_fam/H$(H)η$(η)λ$(lambda[1])T$(n_ep)/" 

if !isdir(out_path)
    println("creating directory")
    mkdir(out_path)
end
for i in 1:length(fs) 
    s = HopfieldDCA.score(m.Ks[i], m.V); PPV_h = HopfieldDCA.compute_PPV(s, structs_dict[fs[2]]);
    @load joinpath(folders_dict[fs[i]], "PPV_s.jld2") ss
    @load joinpath(folders_dict[fs[i]], "PPV_plm.jld2") splm
    @save joinpath(out_path, "parsH$(H)η$(η)λ$(lambda[1])T$(n_ep).jld2") m PPV_h
    
    fig,ax1 = plt.subplots()         
    ax1.plot(ss, label="Stucture")
    ax1.plot(splm, label="PLM")
    ax1.plot(PPV_h, label="Hop")
    ax1.set_xscale("log")
    ax1.set_xlabel("Number of Predictions")
    ax1.set_ylabel("PPV")
    ax1.legend()
    savefig(joinpath(out_path,"ppv$(fs[i])_H$(H)η$(η)λ$(lambda[1])T$(n_ep).png")) 
end
    

In [ ]:
#code for testing analytical gradient
using HopfieldDCA; using Tullio; import Flux: gradient;
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H =32; alg = HopPlmVar_gen(H, filepath;T=Float64); _w = alg.W ./ sum(alg.W); tmp = Stg(alg);
@time gt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@time gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
sum(gt.K[:].-gk[:])
sum(gt.V[:].-gv[:])
import BenchmarkTools: @btime
@btime gkt,gvt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@btime gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
n_ep = 30; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
println("the end")


In [ ]:
structfile = structs_dict[595]
s = HopfieldDCA.score_full(m.K, m.V); PPV_h = HopfieldDCA.compute_PPV(s, structfile);
ssm = HopfieldDCA.score(m1.K, m1.V); PPV_h_sm = HopfieldDCA.compute_PPV(ssm, structfile);
close("all"); loglog(PPV_h_sm, label="H = 21"); loglog(PPV_h, label="H = 32");legend();savefig("ciao.png");



In [ ]:

xx = []; for i in 1:21
    for j in i+1:21
        push!(xx,dot(VV[:,i]./norm(VV[:,i]),VV[:,j]./norm(VV[:,j])))
    end 
end; close("all");hist(xx); savefig("../randV.png")

x = []; for i in 1:21
    for j in i+1:21
        push!(x,dot(m.V[:,i]./norm(m.V[:,i]),m.V[:,j]./norm(m.V[:,j])))
    end 
end; close("all");hist(x); savefig("../ort50_sf_V.png")


In [ ]:

for i in 1:length(Zs)
    fig,ax1 = plt.subplots() 
    ax1.plot(ppv[i], label="Hop")
    ax1.plot(ppvs_fixed_V[i], label="Hop fix V")
    ax1.set_xscale("log")
    ax1.set_xlabel("Number of Predictions")
    ax1.set_ylabel("PPV")
    ax1.legend()
    savefig("../Z$(Zs[i]).png"); 
end


In [ ]:
@load joinpath(folders_dict[f], "PPV_plm.jld2") splm
fig,ax1 = plt.subplots();ax1.plot(ppv, label="Hop");ax1.plot(ppv_V, 
    label="Hop fix V"); ax1.plot(splm, label="Plm"); ax1.set_xscale("log");ax1.set_xlabel("Number of 
    Predictions");ax1.set_ylabel("PPV");ax1.legend();savefig("../$(f)comp.png"); 


fig,ax1 = plt.subplots();ax1.plot(ppv, label="Hop");ax1.plot(ppv_V, 
    label="Hop fix V"); ax1.set_xscale("log");ax1.set_xlabel("Number of 
    Predictions");ax1.set_ylabel("PPV");ax1.legend();savefig("../$(f)comp.png"); 

In [1]:
cc = [100, 500, 1000]; f = 14; fig,ax1 = plt.subplots(); for n_cont in cc
    s = HopfieldDCA.score_full2(m.K, m.V, top_c = n_cont); 
    ppv = compute_PPV(s, structs_dict[f])
    println(n_cont)
    ax1.plot(ppv, label = "CC = $(n_cont)")
end; ax1.plot(PPV_h, label = "Frob_norm"); ax1.set_xscale("log"); ax1.set_xlabel("Number of 
    Predictions"); ax1.set_ylabel("PPV");  ax1.legend(); savefig("../f$(f)cont_anal.png");  
    
    
    

LoadError: UndefVarError: `plt` not defined

In [ ]:
## testing on subsamples
ppvs = []; ppvs_plm = []; f = 14; H = 21; n_ep = 200;  Zs = [10, 50, 100, 500, 1000]; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.5; λ = Float32.(0.001);  @time m,n = trainer_J_fixedV(alg, n_ep, V[:,1:H], η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = false, 
    par_save = false);
    res = plmdca_asym(alg.Z, Float64.(alg.W) ./ sum(Float64.(alg.W)), lambdaJ=0.01,lambdaH=0.01,epsconv=1e-12);
    ppv_plm = compute_PPV(res.score, structs_dict[f]);
    s = score(m.K, V); ppv = compute_PPV(s, structs_dict[f]);
    push!(ppvs, ppv)
    push!(ppvs_plm, ppv_plm)
    
end

fig,ax1 = plt.subplots(); for i in 1:length(Zs)
    ax1.plot(ppvs[i] .- ppvs_plm[i], label="Hop fix V - Plm $(Zs[i])")
end; ax1.set_xscale("log"); ax1.set_xlabel("Number of 
    Predictions"); ax1.set_ylabel("PPV"); ax1.legend(); savefig("../subsample_fixedV$(f).png");

ppvs = []; ppvs_plm = []; f = 14; H = 21; n_ep = 200;  Zs = [10, 50, 100, 500, 1000]; for n_seq in Zs
    alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.5; λ = Float32.(0.001);  #=@time m,n = trainer_J_givenV(alg, n_ep, V, η = η, λ = λ, 
    structfile = structs_dict[f],
    folder = folders_dict[f], 
    savefile = folders_dict[f], 
    fig_save = false, 
    par_save = false);=#
    res = plmdca_asym(alg.Z, Float64.(alg.W) ./ sum(Float64.(alg.W)), lambdaJ=0.01,lambdaH=0.01,epsconv=1e-12);
    ppv_plm = compute_PPV(res.score, structs_dict[f]);
    s = score(m.K, V); ppv = compute_PPV(s, structs_dict[f]);
    push!(ppvs, ppv)
    push!(ppvs_plm, ppv_plm)
    
end

fig,ax1 = plt.subplots(); for i in 1:length(Zs)
    ax1.plot(ppvs[i] .- ppvs_plm[i], label="Hop fix V - Plm $(Zs[i])")
end; ax1.set_xscale("log"); ax1.set_xlabel("Number of 
    Predictions"); ax1.set_ylabel("PPV"); ax1.legend(); savefig("../subsample_fixedV$(f).png");



In [ ]:
ppvs = []; ppvs_plm = []; f = 14; H = 32; n_ep = 1000;  Zs = [100, 500, 1000]; for n_seq in Zs
           alg = HopPlmVar_gen(H, seq_paths_dict[f], n_seq, T = Float32); η = 0.5; λ = 0.001;  @time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
           structfile = structs_dict[f],
           folder = folders_dict[f], 
           savefile = folders_dict[f],
           orthog = false,
           fig_save = false, 
           par_save = false);
           res = plmdca_asym(alg.Z, Float64.(alg.W) ./ sum(Float64.(alg.W)), lambdaJ=0.01,lambdaH=0.01,epsconv=1e-12);
           ppv_plm = compute_PPV(res.score, structs_dict[f]);
           s = score(m.K, m.V); ppv = compute_PPV(s, structs_dict[f]);
           push!(ppvs, ppv)
           push!(ppvs_plm, ppv_plm) 
       end

fig,ax1 = plt.subplots(); for i in 1:length(Zs)
    ax1.plot(ppvs[i], label="Hop fix V $(Zs[i])")
    ax1.plot(ppvs_plm[i], label="Plm $(Zs[i])") 
end; ax1.set_xscale("log"); ax1.set_xlabel("Number of 
    Predictions"); ax1.set_ylabel("PPV"); ax1.legend(); savefig("../subsample_fixedV$(f).png");


In [ ]:
## field test 

en0 = rand(21, 53, size(alg.Z,2)); J = rand(21,53,21,53); en = zeros(21,53,size(alg.Z,2)); h = rand(21, 53); for a in 1:21 for i in 1:53  for m in 1:8871 
       en[a, i, m] = en0[a,i,m] + h[a, i]
        end end end; en2 = en0 .+ h; sum(abs2, en2 .- en)





In [ ]:
using StatsBase, HopfieldDCA

In [ ]:
@time ss = ar_gen(m.K, m.V, m.h, size(alg.Z,2)); fsil = compute_weighted_frequencies(ss, 
    0);  f = compute_weighted_frequencies(Int8.(alg.Z), 0.2);
fig, ax = plt.subplots(); ax.scatter(f[1][:], fsil[1][:], label=
    "$(round(cor(f[1][:], fsil[1][:]), digits = 2))");  ax.set_xlabel("Data"); ax.set_ylabel(
    "Model"); ax.set_title("1-point"); ax.legend();savefig(
    "../new1generative_semi_fields_low.png"); fig, ax = plt.subplots();ax.scatter(f[2][:], fsil[2][:], label=
    "$(round(cor(f[2][:], fsil[2][:]), digits = 2))");ax.set_xlabel("Data"); ax.set_ylabel("Model"
    ); ax.set_title("2-point"); ax.legend(); savefig(
    "../new2generative_semi_fields_low.png")


@time ss = ar_gen(m.K, m.V, size(alg.Z,2)); fsil = compute_weighted_frequencies(ss, 
    0);  f = compute_weighted_frequencies(Int8.(alg.Z), 0.2);
fig, ax = plt.subplots(); ax.scatter(f[1][:], fsil[1][:], label=
    "$(round(cor(f[1][:], fsil[1][:]), digits = 2))");  ax.set_xlabel("Data"); ax.set_ylabel(
    "Model"); ax.set_title("1-point"); ax.legend();savefig(
    "../1generative_semi_nofields.png"); fig, ax = plt.subplots();ax.scatter(f[2][:], fsil[2][:], label=
    "$(round(cor(f[2][:], fsil[2][:]), digits = 2))");ax.set_xlabel("Data"); ax.set_ylabel("Model"
    ); ax.set_title("2-point"); ax.legend(); savefig(
    "../2generative_semi_nofields.png")


@time ss = ar_gen(KK, m.V, m.h, size(alg.Z,2)); fsil = compute_weighted_frequencies(ss, 
    0);  f = compute_weighted_frequencies(Int8.(alg.Z), 0.2);
fig, ax = plt.subplots(); ax.scatter(f[1][:], fsil[1][:], label=
    "$(round(cor(f[1][:], fsil[1][:]), digits = 2))");  ax.set_xlabel("Data"); ax.set_ylabel(
    "Model"); ax.set_title("1-point"); ax.legend();savefig(
    "../full_1generative_fields.png"); fig, ax = plt.subplots();ax.scatter(f[2][:], fsil[2][:], label=
    "$(round(cor(f[2][:], fsil[2][:]), digits = 2))");ax.set_xlabel("Data"); ax.set_ylabel("Model"
    ); ax.set_title("2-point"); ax.legend(); savefig(
    "../full2generative_fields.png")



In [ ]:

@time Z = ar_gen(m.K, m.V, m.h, 20000); w = compute_weights(Int8.(Z), 0)[1]; _w = w ./ sum(w); @load "../DataAttentionDCA/data/PF00014/PPV_plm.jld2"; res = plmdca_asym(
    Z, _w, lambdaJ=0.01,lambdaH=0.01,epsconv=1e-12); ppv_ar_hop = compute_PPV(res.score, structs_dict[f]);
fig,ax1 = plt.subplots(); ax1.plot(splm, label="Plm"); ax1.plot(
           ppv_ar_hop, label="Full_Hop ar 2%"); ax1.set_xscale("log"); ax1.set_xlabel(
           "Number of Predictions"); ax1.set_ylabel("PPV"); ax1.legend(); savefig("../f14_semi_ar_hop_2%ppv.png")


In [ ]:
f1, f2 = compute_weighted_frequencies(Int8.(Z), 0.2); c = zeros(53*20, 53*20); for i in 1:53 
    for j in 1:53 
        for a in 1:20
            for b in 1:20
                c[20*(i-1)+a, 20*(j-1)+b] = f2[20*(i-1)+a, 20*(j-1)+b] - f1[20*(i-1)+a]*f1[20*(j-1)+b]
            end
        end
    end
end
     


n_ep = 20; f =21; η = 0.005; λ = 0.001; m, s = trainer_hop(n_ep, xi, η = η,  λ = λ, structfile = structs_dict[f])
